In [13]:
import os
import pickle
import torch
import datetime
import numpy as np
from math import ceil
from model_vc import Generator


def pad_seq(x, base=32):
    len_out = int(base * ceil(float(x.shape[0])/base))
    len_pad = len_out - x.shape[0]
    assert len_pad >= 0
    return np.pad(x, ((0,len_pad),(0,0)), 'constant'), len_pad

# device = 'cuda:0'
device = 0
uttr = '275'
baseline = False
spk_emb = 'res' # dvec, xvec, res

# load in models
if baseline:
    G = Generator(32,256,512,32,None).eval().to(device) # autovc.ckpt model shape
    autovc_model = '/home/ytang363/7100_voiceConversion/pretrain/autovc.ckpt'
else:
    # G = Generator(16,256,512,16).eval().to(device) # oct-16 model shape
    # G = Generator(128, 256, 512, 32, 512).eval().to(device) # xvec model edit first parameter to change emb dimension
    G = Generator(32, 256, 512, 32, None).eval().to(device)
    autovc_model = '/home/ytang363/7100_voiceConversion/model_res_32/model_checkpoint_res_32_1500000.ckpt'

g_checkpoint = torch.load(autovc_model, map_location=torch.device(device))
print(g_checkpoint.keys())
G.load_state_dict(g_checkpoint['model'])

# metadata contains [filename, speaker embedding, spectrogram]
metadata = pickle.load(open(f'/home/ytang363/7100_voiceConversion/metadata/metadata_test/{spk_emb}_metadata_{uttr}.pkl', "rb")) # metadata_002_xvec
spect_vc = []   

dict_keys(['model', 'optimizer'])


In [14]:
for sbmt_i in metadata:
    print(f'source speaker: {sbmt_i[0]}')

    x_org = sbmt_i[2] # spectrogram
    x_org, len_pad = pad_seq(x_org)
    

    uttr_org = torch.from_numpy(x_org[np.newaxis, :, :]).to(device) # spectrogram
    emb_org = torch.from_numpy(sbmt_i[1][np.newaxis, :]).to(device) # speaker embedding
    # print(emb_org.shape)
    print(uttr_org.shape)
    
    for sbmt_j in metadata:
        print(f'target speaker: {sbmt_j[0]}')
        
        emb_trg = torch.from_numpy(sbmt_j[1][np.newaxis, :]).to(device)
        
        with torch.no_grad():
            _, x_identic_psnt, _ = G(uttr_org, emb_org, emb_trg)
            
        if len_pad == 0:
            uttr_trg = x_identic_psnt[0, 0, :, :].cpu().numpy()
        else: # excluding a padding portion
            uttr_trg = x_identic_psnt[0, 0, :-len_pad, :].cpu().numpy()
        
        spect_vc.append( ('{}x{}'.format(sbmt_i[0], sbmt_j[0]), uttr_trg) )
        # print(spect_vc)

current_date = datetime.datetime.now()
formatted_date = current_date.strftime("%m-%d")

if baseline:
    outName = f'{uttr}_baseline-results.pkl'
else:
    # outName = f'{uttr}_retrained-results.pkl'
    outName = f'{uttr}_res_32_1500k-results.pkl'

# Output dump
path_dir = '/home/ytang363/7100_voiceConversion/results/result_test/res_32'
with open(os.path.join(path_dir, outName), 'wb') as handle:
    pickle.dump(spect_vc, handle)

source speaker: p231
torch.Size([1, 192, 80])
target speaker: p231
target speaker: p243
target speaker: p272
target speaker: p279
target speaker: p314
target speaker: p339
source speaker: p243
torch.Size([1, 288, 80])
target speaker: p231
target speaker: p243
target speaker: p272
target speaker: p279
target speaker: p314
target speaker: p339
source speaker: p272
torch.Size([1, 160, 80])
target speaker: p231
target speaker: p243
target speaker: p272
target speaker: p279
target speaker: p314
target speaker: p339
source speaker: p279
torch.Size([1, 128, 80])
target speaker: p231
target speaker: p243
target speaker: p272
target speaker: p279
target speaker: p314
target speaker: p339
source speaker: p314
torch.Size([1, 192, 80])
target speaker: p231
target speaker: p243
target speaker: p272
target speaker: p279
target speaker: p314
target speaker: p339
source speaker: p339
torch.Size([1, 224, 80])
target speaker: p231
target speaker: p243
target speaker: p272
target speaker: p279
target spe